## Scraping Property Access PH through Selenium
This notebook was prepared by Adem Inovejas, Christopher Lim, Czarina Tiu, and Uriel Grace Magtibay (students of DATA102 S11 Y2022-2023).  

In your notebook, make sure you have the following details outlined:
- The website scraped
- Date and time when the data was collected
- What were the challenges encountered? You may narrate or illustrate this in the notebook.
- Do you think the collected data contains any personally identifiable information (PII)?
- Conclude with your key learnings and findings.

## Importing Libaries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import datetime
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Setup

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = "https://propertyaccess.ph/offer/sale"
driver.get(url)

## Data Collection

### Collecting  the URLs of each properties

The website is structured with multiple pages with each page having approximately 20 property listings. Each property listing is shown on the main page as a clickable widget, that when clicked, will redirect to a new page with more information regarding the property listing. So the first step in our data collection process was to collect the url of each property across the website. 

In [4]:
has_pages = True
pages = 1
urls = []
total_time = 0
while(has_pages):  
    start = time.perf_counter()
    
    products_container = driver.find_element(by="xpath", value='//div[@class="list-product page-content"]')
    
    # gets the list of products
    items = products_container.find_elements(by="xpath", value='.//div[@class="product-wrapper"]')
    for item in items:
        # get the href of the anchor of "product-img"
        url = item.find_element(by="xpath", value='.//div[@class="product-img"]/a').get_attribute('href')
        date_published = item.find_element(by="xpath", value='.//div[@class="date"]').text
        urls.append((url, date_published))
    
    end = time.perf_counter()
    
    total_time += end - start
    print('Extracted Page %d | %d items found | %.2fs' % (pages, len(items), end - start))
    try:
        # checks if next page button is available by getting the xpath
        next_page = driver.find_element("xpath", value='//ul[@class="pargination"]/li[@class="next flex-end"]/a').get_attribute("href");
        driver.get(next_page)
        pages += 1
    except:
        # if next page button is not available, stop the loop
        has_pages = False

print('Number of pages:', pages)
print('Number of products:', len(urls))
print('Average Time per page:', total_time / len(urls))
print('Total Time:', total_time)

Extracted Page 1 | 20 items found | 0.57s
Extracted Page 2 | 20 items found | 0.58s
Extracted Page 3 | 20 items found | 0.60s
Extracted Page 4 | 20 items found | 0.62s
Extracted Page 5 | 20 items found | 0.74s
Extracted Page 6 | 20 items found | 0.72s
Extracted Page 7 | 20 items found | 0.75s
Extracted Page 8 | 20 items found | 0.68s
Extracted Page 9 | 20 items found | 0.70s
Extracted Page 10 | 20 items found | 0.71s
Extracted Page 11 | 20 items found | 0.70s
Extracted Page 12 | 20 items found | 0.75s
Extracted Page 13 | 20 items found | 0.67s
Extracted Page 14 | 20 items found | 0.74s
Extracted Page 15 | 20 items found | 0.73s
Extracted Page 16 | 20 items found | 0.67s
Extracted Page 17 | 20 items found | 0.69s
Extracted Page 18 | 20 items found | 0.69s
Extracted Page 19 | 20 items found | 0.68s
Extracted Page 20 | 20 items found | 0.73s
Extracted Page 21 | 20 items found | 0.65s
Extracted Page 22 | 20 items found | 0.69s
Extracted Page 23 | 20 items found | 0.66s
Extracted Page 24 | 

Exporting the URLs through pandas dataframe (as .csv)

In [6]:
df = pd.DataFrame(urls, columns=['URL', 'Date Published'])
df.head()

,URL,Date Published
0,https://propertyaccess.ph/property/3-br-condo-...,Published on: 30/03/2022
1,https://propertyaccess.ph/property/1br-condo-i...,Published on: 30/03/2022
2,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
3,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
4,https://propertyaccess.ph/property/2br-condo-i...,Published on: 1/06/2022


In [7]:
df.to_csv('property_urls.csv', index=False)

### Going through each property page to get the details

With the urls obtained in the previous step, each url will be scrapped for information about the property. The data obtained in this step are property details (name, address, price, author), utility information (bedroom, showers, furnish, parking, lot area), features, and the timestamp that notes what date and time the information was collected


In [4]:
df = pd.read_csv('property_urls.csv')
df.head()

,URL,Date Published
0,https://propertyaccess.ph/property/3-br-condo-...,Published on: 30/03/2022
1,https://propertyaccess.ph/property/1br-condo-i...,Published on: 30/03/2022
2,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
3,https://propertyaccess.ph/property/3-bedroom-c...,Published on: 17/07/2022
4,https://propertyaccess.ph/property/2br-condo-i...,Published on: 1/06/2022


In [5]:
data = []
total_time = 0
for i, item in enumerate(list(zip(df['URL'], df['Date Published']))):
    try:
        start = time.perf_counter()
        url, date_published = item
        driver.get(url)

        # get the container which contains the property details
        property_details = driver.find_element(by="xpath", value='//div[@class="product-detail-wraper"]')

        # find the name
        name = property_details.find_element(by="xpath", value='.//h1[@class="basic-info__name"]').text
        address = property_details.find_element(by="xpath", value='.//div[@class="basic-info__street"]').text
        price = property_details.find_element(by="xpath", value='.//div[@class="basic-info__price"]').text
        author = property_details.find_element(by="xpath", value='.//div[@class="agent-name line-clamp lc-2"]').text

        # get the bedrooms, showers, parking, furnish type, total developed and lot area through utility info
        utility_info = property_details.find_element(by="xpath", value='.//div[@class="basic-info__utilities"]')

        try:
            bedrooms = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/f24d0f0d48276caeca2060fd60abf269.svg"]/..').text
        except:
            bedrooms = None

        try:
            showers = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/a6e500c94a1eaacdd59fde045fba89f0.svg"]/..').text
        except:
            showers = None

        try:
            furnish = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/9fb04d2eb34997da34e3dcfd140f51b6.svg"]/..').text
        except:
            furnish = None

        try:
            parking = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/33d487e03cfed78b821458ab3c39b574.svg"]/..').text
        except:
            parking = None

        try:
            total_developed = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/57cebe2c8bcb0aa7f604503ddb740621.svg"]/..').text
        except:
            total_developed = None

        try:
            lot_area = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.6/da2ffe9d4238be9a3d697b4195b64fe7.svg"]/..').text
        except:
            lot_area = None

        # if section title is Property Features or Amenities
        features = [feature.text for feature in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Property Features" or text()="Amenities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Facilities
        facilities = [facility.text for facility in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Facilities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Nearby Places
        nearby_places = [place.text for place in property_details.find_elements(by="xpath", value='.//section[@class="product-nearby"]//div[@class="item-title"]')]

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        end = time.perf_counter()

        total_time += end - start
        
        print('[%4d] %s | %.2fs' % (i, name, end - start))

        data.append((name, address, author, price, bedrooms, showers, parking, furnish, total_developed, lot_area, features, facilities, nearby_places, url, timestamp))
    except:
        # if page is un-available
        print('Error occured on', url)

[   0] 3 BR Condo in The Grand Midori Ortigas, Pasig | 1.72s
[   1] 1 Bedroom Condo in The Grand Midori Ortigas, Pasig | 1.26s
[   2] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.55s
[   3] 3 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.56s
[   4] 2BR Condo in Residences at The Galleon, Pasig | 1.91s
[   5] Penthouse in Residences at The Galleon, Pasig | 2.93s
[   6] 3 Bedroom Condo in Aurelia Residences, Taguig | 2.07s
[   7] 3 Bedroom Condo in Valencia Hills Tower E, Quezon City | 2.41s
[   8] 2 Bedroom Condo in Valencia Hills Tower E, Quezon City | 1.35s
[   9] 1 Bedroom Condo in Valencia Hills Tower E, Quezon City | 1.16s
[  10] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.96s
[  11] 2 Bedroom Condo in The Rise Makati, Makati | 2.06s
[  12] 3 Bedroom Condo in Aurelia Residences, Taguig | 1.48s
[  13] 2 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.52s
[  14] 1 Bedroom Condo in Shang Residences at Wack Wack, Mandaluyong | 1.42s
[  15] 1

[ 138] 2 Bedroom House & Lot, Cebu City | 1.13s
[ 139] 1 Bedroom Condo, Cebu City | 1.23s
[ 140] 4 Bedroom House & Lot, Cebu City | 1.12s
[ 141] 1 Bedroom Condo, Cebu City | 1.12s
[ 142] 1 Bedroom Condo in 32 Sanson, Cebu City | 1.44s
[ 143] 3 Bedroom House & Lot, Consolacion | 1.35s
[ 144] 4 Bedroom House & Lot, Talisay | 1.16s
[ 145] Studio Condo in Avida Towers Riala, Cebu City | 1.39s
[ 146] Studio Condo Sunvida Tower, Cebu City | 0.99s
[ 147] 5 Bedroom House & Lot, Antipolo | 1.30s
[ 148] 1 Bedroom Condo in San Lorenzo Place, Makati | 1.30s
[ 149] 7 Bedroom House & Lot, Silang | 1.26s
[ 150] 5 Bedroom House & Lot, Las Piñas | 1.10s
[ 151] 2 Bedroom Condo in The Balmori Suites, Makati | 1.51s
[ 152] 6 Bedroom House & Lot, Quezon City | 1.17s
[ 153] 9 Bedroom House & Lot, Makati | 1.10s
[ 154] 4 Bedroom Condo in St. Mortiz Private Estate, Manila | 1.10s
[ 155] 2 Bedroom Condo in Icon Plaza, Taguig | 1.57s
[ 156] 1 Bedroom Condo in Joya Lofts and Towers, Makati | 1.19s
[ 157] 3 Bedro

[ 296] 7 Bedroom House & Lot, Quezon City | 1.21s
[ 297] 4 Bedroom House & Lot, San Juan | 1.58s
[ 298] 4 Bedroom House & Lot, Pasig | 1.10s
[ 299] 3 Bedroom House & Lot, Pasig | 1.23s
[ 300] 5 Bedroom House & Lot, Pasig | 1.19s
[ 301] 5 Bedroom House & Lot, Quezon City | 1.19s
[ 302] 5 Bedroom House & Lot, Quezon City | 1.25s
[ 303] 4 Bedroom House & Lot, Manila | 1.12s
[ 304] 4 Bedroom House & Lot, Quezon City | 1.05s
[ 305] 4 Bedroom House & Lot, Quezon City | 1.19s
[ 306] 4 Bedroom House & Lot, Quezon City | 1.11s
[ 307] Commercial Space, Pangil | 1.47s
[ 308] 2 Bedroom Condo in Venice Luxury Residences, Taguig | 1.41s
[ 309] 3 Bedroom House & Lot, Taguig | 1.29s
[ 310] Studio Condo in Adriatico Place, Manila | 1.41s
[ 311] 1 Bedroom Condo in Venice Luxury Residences, Taguig | 1.40s
[ 312] 1 Bedroom Condo For Sale in Fifth Avenue Place, BGC, Taguig | 1.43s
[ 313] 2 Bedroom House & Lot, Biñan | 1.24s
[ 314] 2 Bedroom House & Lot, Batangas | 1.25s
[ 315] Residential Lot, Antipolo | 1

[ 456] 5 Bedroom House & Lot, Las Piñas | 1.16s
[ 457] 2 Bedroom Condo in One Uptown Residence, Taguig | 1.43s
[ 458] 3 Bedroom House & Lot, Malolos | 1.27s
[ 459] Residential Lot, Calabarzon | 1.15s
[ 460] 1 Bedroom Condo in The Maridien, Taguig | 1.47s
[ 461] 2 Bedroom Condo, Parañaque | 1.18s
[ 462] 4 Bedroom House & Lot, Taytay | 1.51s
[ 463] 5+ Bedroom House & Lot, Muntinlupa | 1.12s
[ 464] 1 Bedroom Condo in The Beaufort, Taguig | 1.31s
[ 465] 1 Bedroom Unit at The Velaris, Pasig | 1.43s
[ 466] 2 Bedroom Condo in Acqua Private Residences, Mandaluyong | 1.57s
[ 467] 2 Bedroom Condo in Icon Residences, Taguig | 1.54s
[ 468] 2 Bedroom Condo in Avida Towers Verte, Taguig | 1.36s
[ 469] 2 Bedroom Condo in Panglao Oasis, Taguig | 1.26s
[ 470] Residential Lot, Antipolo | 1.12s
[ 471] 4 Bedroom House & Lot, Taytay | 1.16s
[ 472] 2 Bedroom House & Lot in Cabanatuan City | 1.17s
[ 473] 1 Bedroom Condo in Forbeswood Heights, Taguig | 1.56s
[ 474] 4 Bedroom House & Lot, Quezon City | 1.33s
[

[ 612] 3 Bedroom Condo in The Suites, Taguig | 1.65s
[ 613] Studio Condo, Quezon City | 1.33s
[ 614] 1 Bedroom Condo in Andrea North, Quezon City | 1.61s
[ 615] Studio Condo in Patio Madrigal, Pasay | 1.71s
[ 616] 3 Bedroom House & Lot, Cabanatuan City | 1.46s
[ 617] 3 Bedroom Condo in Park Mckinley West, Manila | 1.52s
[ 618] 2 Bedroom Condo in Circulo Verde, Quezon City | 2.23s
[ 619] 3 Bedroom House & Lot, Imus | 1.50s
[ 620] 3 Bedroom House & Lot, General Trias | 1.22s
[ 621] 3 Bedroom House & Lot, General Trias | 1.24s
[ 622] 3 Bedroom Condo in Two Serendra, Taguig | 1.29s
[ 623] Residential Lot, Tagaytay | 1.14s
[ 624] Residential Lot, Tagaytay | 1.17s
[ 625] Residential Lot, Tagaytay | 2.09s
[ 626] 3 Bedroom Condo in Mosaic Tower, Makati | 1.40s
[ 627] 4 Bedroom House & Lot, Parañaque | 1.11s
[ 628] Residential Lot, Calamba | 1.12s
[ 629] 3 Bedroom Condo in Venice Luxury Residences, Taguig | 1.49s
[ 630] Residential Lot, Porac | 0.97s
[ 631] 3 Bedroom Condo in Pacific Plaza Towe

...

[1977] 2 BR Condo in The ADB Avenue Tower, Pasig City | 0.96s
[1978] 4 BR Condo in AIC Gold Ortigas Center, Pasig City | 0.92s
[1979] 2 BR Condo in St Francis Tower 2, Pasig City | 1.95s
[1980] 5 BR House in Kapitolyo, Pasig City | 0.96s
[1981] 3 BR House in Kapitolyo, Pasig City | 1.13s
[1982] 3 BR House in Kapitolyo, Pasig City | 0.82s
[1983] 4 BR House in Kapitolyo, Pasig City | 0.87s
[1984] 3 BR House in Kapitolyo, Pasig City | 0.89s
[1985] 3 BR House in Kapitolyo, Pasig City | 0.99s
[1986] 3 BR House in Kapitolyo, Pasig City | 0.93s
[1987] 35 SQM Studio with Patio Condominium in Pioneer Woodlands, Mandaluyong | 1.23s
[1988] 1 Bedroom Condo with Parking in Makati | 1.02s
[1989] 2 BR Condo in Chino Roces, Makati City | 1.35s
[1990] 2 Bedroom Condominium in San Lorenzo Place Chino Roces, Makati City | 1.23s
[1991] 3 Bedroom House and Lot in Ponticelli Gardens, Daang Hari, Bacoor, Cavite | 1.01s
[1992] Pre-Selling 3 Bedroom House and Lot for sale in Santa Rosa, Laguna | Valenza by Cro

[2103] 1 Bedroom Unit with Balcony at SMDC Air Residences, Makati | 1.24s
[2104] 1 Bedroom Unit at SMDC Sands Residences, Pasay | 1.12s
[2105] Studio unit in Amaia Skies Avenida, Manila | 1.18s
[2106] 2 Bedroom Condominium in Pine Suites Tagaytay | 1.14s
[2107] House in Brookside Hills Cainta, Rizal | 0.89s
[2108] Junior 1 Bedroom Unit in Avida Towers Cloverleaf | 1.42s
[2109] 2 BR Condo in Avida Towers Cloverleaf Tower 2 | 1.14s
[2110] Executive Studio Unit with Balcony for Sale in St. Mark Residences McKinley Hill | 1.10s
[2111] 2BR Townhouse in Cabanatuan City | 0.98s
[2112] House in Filinvest Homes Antipolo, Rizal | 1.04s
[2113] Studio Unit Condominium in Pine Suites,Tagaytay | 1.06s
[2114] 1 Bedroom Unit with Balcony & Den in SMDC Red Residences, Makati | 1.33s
[2115] 1 Bedroom Unit in SMDC Sail Residences, Pasay | 1.21s
[2116] 2 Bedroom Unit with Balcony in SMDC Light 2 | 1.27s
[2117] 1 Bedroom Unit with Balcony in SMDC Glam Residences | 1.37s
[2118] 1 Bedroom in Gold Residences,

[2232] 2 BR House in Amore at Portofino, Las Pinas | 1.16s
[2233] 2 BR Condominium unit in Florence Mckinley Hill, Taguig | 1.27s
[2234] 3 BR Condominium in One McKinley Place, Taguig | 1.13s
[2235] 3 Bedroom Penthouse Condominium Unit in St. Francis Shangri-La Place, Pasig City | 1.14s
[2236] 3 Bedroom Townhouse in New Manila, Quezon City | 1.17s
[2237] 3 BR Condo in Olympic Heights, Quezon City | 1.30s
[2238] 3 Bedroom Townhouse in San Juan | 0.98s
[2239] 3 Bedroom Condominium in Pine Crest, Quezon City | 1.41s
[2240] 3 Bedroom Condominium in Sheridan Towers, Mandaluyong | 1.28s
[2241] 2 Bedroom Condominium in Shell Residences, Pasay City | 1.36s
[2242] 1 Bedroom Condominium in The Radiance Manila Bay, North Tower, Pasay City | 1.39s
[2243] 2 Bedroom Condominium in The Lerato , Tower 3, Makati City | 1.51s
[2244] 2 Bedroom Condominium in The Proscenium - Sakura Tower, Makati City | 1.34s
[2245] 2 Bedroom Condominium in San Lorenzo Place, Makati City | 1.38s
[2246] 2 Bedroom Condomini

[2365] 1 BR Condominium Unit in Avant at The Fort, Taguig | 1.28s
[2366] 1 BR Condominium Unit in Bellagio 1, Taguig | 1.15s
[2367] 1 BR Condominium Unit in Forbeswood Heights, Taguig | 1.12s
[2368] 1 BR Condominium Unit in Avida Towers 9th Avenue, Taguig | 1.21s
[2369] 1 BR Condominium Unit in Bellagio 3, Taguig | 1.67s
[2370] 5 BR House and Lot in Ayala Alabang - Muntinlupa | 1.11s
[2371] 6 BR House and Lot in Ayala Alabang - Muntinlupa | 1.10s
[2372] 4 BR House and Lot in Ayala Alabang - Muntinlupa | 1.11s
[2373] 6 BR House and Lot in Ayala Alabang - Muntinlupa | 1.24s
[2374] 5 BR House and Lot in Ayala Alabang - Muntinlupa | 1.04s
[2375] 6 BR House and Lot in BF Northwest BF Homes Parañaque - Parañaque | 1.10s
[2376] 4 BR House and Lot in BF Homes Las Piñas - Las Piñas | 1.19s
[2377] 5 BR House and Lot in United Parañaque Subdivision 4 - Parañaque | 1.13s
[2378] 1 BR Unit in Ilustrata Residences, Quezon City | 1.33s
[2379] 3 BR House and Lot in Katarungan Village, Muntinlupa | 1.11

[2488] Studio Type Condominium in St. Francis Shangri-La Place, Mandaluyong | 1.19s
[2489] 1 Bedroom Condominium in Wack-Wack Twin Towers, Mandaluyong | 1.09s
[2490] 2 BR Condo Unit in The St. Francis Shangri-La Place | 1.15s
[2491] 1 Bedroom Condominium in Twin Oaks Place, Mandaluyong | 1.39s
[2492] 1 BR Condominium unit in Wack Wack Twin Tower, Mandaluyong | 1.02s
[2493] 1 BR Condominium unit in Light Residences SMDC, Mandaluyong | 1.45s
[2494] Three Bedrooms at The Address at Wack Wack, Mandaluyong | 1.15s
[2495] Two Bedrooms at The St. Francis Shangri-La Place, Tower 1 | 1.28s
[2496] 4 Bedroom House and Lot in Marikina City | 1.16s


In [6]:
# print the total time it took to scrape
print('Total Number of Items: %d' % len(data))
print('Average Time per Item: %.2f' % (total_time / len(data)))
print('Total Time: %.2f' % total_time)

Total Number of Items: 2491
Average Time per Item: 1.34
Total Time: 3331.11


In [7]:
column_names = ['Name', 'Address', 'Author', 'Price', 'Bedrooms', 'Showers', 'Parking', 'Furnish', 'Total Developed', 'Lot Area', 'Features', 'Facilities', 'Nearby Places', 'URL', 'Timestamp']
df = pd.DataFrame(data, columns = column_names)
df.head()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 BR Condo in The Grand Midori Ortigas, Pasig",,"Federal Land, Inc.","Sale: from ₱ 29,700,000",3,3,2,Unfurnished,from 107 sqm,None,"[Ensuite, Range Hood, Stove/Burner]","[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/3-br-condo-...,2022-10-23 12:25:33
1,"1 Bedroom Condo in The Grand Midori Ortigas, P...",,"Federal Land, Inc.","Sale: from ₱ 10,200,000",1,1,0,Unfurnished,from 46 sqm,None,[Fiber ready],"[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/1br-condo-i...,2022-10-23 12:25:35
2,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,"Sale: from ₱ 107,300,000",3,4,2,Unfurnished,from 242 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-23 12:25:36
3,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,"Sale: from ₱ 54,500,000",3,4,3,Unfurnished,from 231 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-23 12:25:38
4,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,"Sale: from ₱ 41,500,000",2,2,2,Unfurnished,from 109 sqm,None,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-10-23 12:25:40


In [8]:
df.describe()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
count,2491,2491,2491,2491,2327,2327,2337,2318,2490,945,2491,2491,2491,2491,2491
unique,1486,588,320,1020,11,9,9,3,502,415,628,458,11,2491,2477
top,"4 Bedroom House & Lot, Quezon City","Taguig, Metro Manila",Raimund,"Sale: ₱ 15,000,000",2,1,0,Unfurnished,from 107 sqm,280 sqm,[],[],[],https://propertyaccess.ph/property/3-br-condo-...,2022-10-23 13:14:48
freq,53,104,162,41,499,954,728,950,129,23,169,1197,1431,1,2


In [10]:
df.to_json('unclean_data.json', orient='records', indent=2)

In [11]:
driver.close()

## Data Cleaning

In [14]:
data = pd.read_json('unclean_data.json')
data

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 BR Condo in The Grand Midori Ortigas, Pasig",,"Federal Land, Inc.","Sale: from ₱ 29,700,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Ensuite, Range Hood, Stove/Burner]","[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/3-br-condo-...,2022-10-17 17:39:48
1,"1 Bedroom Condo in The Grand Midori Ortigas, P...",,"Federal Land, Inc.","Sale: from ₱ 10,200,000",NaN,NaN,NaN,NaN,NaN,NaN,[Fiber ready],"[Conference Room, Entertainment Area, Function...","[School, Hospital]",https://propertyaccess.ph/property/1br-condo-i...,2022-10-17 17:39:50
2,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,"Sale: from ₱ 107,300,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-17 17:39:51
3,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,"Sale: from ₱ 54,500,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-10-17 17:39:53
4,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,"Sale: from ₱ 41,500,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-10-17 17:39:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,"1 BR Condominium unit in Wack Wack Twin Tower,...",Mandaluyong,Pamela,"Sale: ₱ 5,100,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Balcony, Ensuite, F...","[Security, CCTV, Conference Room, Fitness Cent...",[],https://propertyaccess.ph/property/1-br-condom...,2022-10-17 18:36:44
2447,1 BR Condominium unit in Light Residences SMDC...,"Madison, Mandaluyong, Metro Manila",Maricar,"Sale: ₱ 4,375,700",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Built-in wardrobes,...","[Connected to mall, Fitness Center, Function R...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/1-br-condom...,2022-10-17 18:36:45
2448,"Three Bedrooms at The Address at Wack Wack, Ma...","Old Wack Wack Road, Mandaluyong, Metro Manila",Marie Paz,"Sale: ₱ 23,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Built-in wardrobes]","[Security, CCTV]","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/three-bedro...,2022-10-17 18:36:46
2449,Two Bedrooms at The St. Francis Shangri-La Pla...,"Internal Avenue, Mandaluyong, Metro Manila",Marie Paz,"Sale: ₱ 26,000,000",NaN,NaN,NaN,NaN,NaN,NaN,"[Central air conditioning, Built-in wardrobes,...","[Security, CCTV, Fitness Center, Garden/Lanai,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/two-bedroom...,2022-10-17 18:36:48


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2451 entries, 0 to 2450
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Name             2451 non-null   object        
 1   Address          2451 non-null   object        
 2   Author           2451 non-null   object        
 3   Price            2451 non-null   object        
 4   Bedrooms         0 non-null      float64       
 5   Showers          0 non-null      float64       
 6   Parking          0 non-null      float64       
 7   Furnish          0 non-null      float64       
 8   Total Developed  0 non-null      float64       
 9   Lot Area         0 non-null      float64       
 10  Features         2451 non-null   object        
 11  Facilities       2451 non-null   object        
 12  Nearby Places    2451 non-null   object        
 13  URL              2451 non-null   object        
 14  Timestamp        2451 non-null   datetim

### Price

In [16]:
data['Price'] = data['Price'].str.replace('Sale: from ₱ ','')
data['Price'] = data['Price'].str.replace('Sale: ₱ ','')

In [17]:
data['Price'] = data['Price'].str.replace(',','').astype(int)

In [19]:
data['Price']

0        29700000
1        10200000
2       107300000
3        54500000
4        41500000
          ...    
2446      5100000
2447      4375700
2448     23000000
2449     26000000
2450     26800000
Name: Price, Length: 2451, dtype: int32

### Lot Area and Total Developed

In [20]:
def clean_area(val):
    return float(str(val.replace('from', '').replace('sqm', ''))) if val == val else None

data['Lot Area'] = data['Lot Area'].apply(clean_area)
data['Total Developed'] = data['Total Developed'].apply(clean_area)

In [21]:
data['Lot Area'].value_counts()

Series([], Name: Lot Area, dtype: int64)

In [22]:
data['Total Developed'].value_counts()

Series([], Name: Total Developed, dtype: int64)

### Timestamp

In [23]:
data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.date

In [24]:
data['Timestamp']

0       2022-10-17
1       2022-10-17
2       2022-10-17
3       2022-10-17
4       2022-10-17
           ...    
2446    2022-10-17
2447    2022-10-17
2448    2022-10-17
2449    2022-10-17
2450    2022-10-17
Name: Timestamp, Length: 2451, dtype: object

### Checking missing values per column

In [25]:
# Checking null values per column
data.isna().sum()

Name                  0
Address               0
Author                0
Price                 0
Bedrooms           2451
Showers            2451
Parking            2451
Furnish            2451
Total Developed    2451
Lot Area           2451
Features              0
Facilities            0
Nearby Places         0
URL                   0
Timestamp             0
dtype: int64

In [26]:
data = data.dropna()
data.isna().sum()

Name               0.0
Address            0.0
Author             0.0
Price              0.0
Bedrooms           0.0
Showers            0.0
Parking            0.0
Furnish            0.0
Total Developed    0.0
Lot Area           0.0
Features           0.0
Facilities         0.0
Nearby Places      0.0
URL                0.0
Timestamp          0.0
dtype: float64

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             0 non-null      object 
 1   Address          0 non-null      object 
 2   Author           0 non-null      object 
 3   Price            0 non-null      int32  
 4   Bedrooms         0 non-null      float64
 5   Showers          0 non-null      float64
 6   Parking          0 non-null      float64
 7   Furnish          0 non-null      float64
 8   Total Developed  0 non-null      object 
 9   Lot Area         0 non-null      object 
 10  Features         0 non-null      object 
 11  Facilities       0 non-null      object 
 12  Nearby Places    0 non-null      object 
 13  URL              0 non-null      object 
 14  Timestamp        0 non-null      object 
dtypes: float64(4), int32(1), object(10)
memory usage: 0.0+ bytes


In [28]:
data.dtypes

Name                object
Address             object
Author              object
Price                int32
Bedrooms           float64
Showers            float64
Parking            float64
Furnish            float64
Total Developed     object
Lot Area            object
Features            object
Facilities          object
Nearby Places       object
URL                 object
Timestamp           object
dtype: object

In [18]:
data['Showers'] = data['Showers'].str.replace('+','')
data['Parking'] = data['Parking'].str.replace('+','')
data['Bedrooms'] = data['Bedrooms'].str.replace('+','')

/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Showers'] = data['Showers'].str.replace('+','')
/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Parking'] = data['Parking'].str.replace('+','')
/var/folders/b5/583sbyw12yqf8bss252d8ndc0000gn/T/ipykernel_46913/1857973885.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Bedrooms'] = data['Bedrooms'

In [19]:
data['Showers'] = data['Showers'].astype(float)
data['Parking'] = data['Parking'].astype(float)
data['Bedrooms'] = data['Bedrooms'].astype(float)
# NOTE that int cant be used because of some values with decimals
# for situations with more than 5 of these categories, + is just used. 
# So it was removed in the code above to make it into a float
# please check error mssg

ValueError: could not convert string to float: 'Studio'

In [48]:
# converting unfurnished and furnished to boolean
data['Furnish'] = data['Furnish'].replace({'Unfurnished':0,'Furnished':1,'Semi-furnished':2})
data['Furnish'] = data['Furnish'].astype(int)

In [49]:
data.dtypes

Name                object
Address             object
Author              object
Price                int64
Bedrooms           float64
Showers            float64
Parking            float64
Furnish              int64
Total Developed      int64
Lot Area             int64
Features            object
Facilities          object
Nearby Places       object
URL                 object
Timestamp           object
dtype: object

In [50]:
data.head()

,Name,Address,Author,Price,Bedrooms,Showers,Parking,Furnish,Total Developed,Lot Area,Features,Facilities,Nearby Places,URL,Timestamp
0,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,107300000,4.0,4.0,4.0,0,242,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27
1,3 Bedroom Condo in Shang Residences at Wack Wa...,"Wack Wack Road, Mandaluyong, Metro Manila",Shang Properties,54500000,4.0,4.0,4.0,0,231,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Club House, Entertainment Are...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27
2,"2BR Condo in Residences at The Galleon, Pasig",,Ortigas Land,41500000,2.0,2.0,2.0,0,109,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/2br-condo-i...,2022-09-27
3,"Penthouse in Residences at The Galleon, Pasig","ADB Avenue, Pasig, Metro Manila",Ortigas Land,111500000,4.0,4.0,4.0,0,268,0,"[Central air conditioning, Balcony, Built-in w...","[Security, CCTV, Clubhouse, Connected to mall,...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/penthouse-i...,2022-09-27
4,"3 Bedroom Condo in Aurelia Residences, Taguig","McKinley Parkway, Taguig, Metro Manila",Shang Properties,181302240,4.0,4.0,4.0,0,337,0,[Central air conditioning],"[Security, CCTV, Entertainment Area, Fitness C...","[School, Hospital, Mall, Transportation hub]",https://propertyaccess.ph/property/3-bedroom-c...,2022-09-27


In [51]:
data.to_json('cleaned data.json',orient='columns')

In [ ]:
f = data['Features']
unique_list = []
for x in f: 
    for i in x:
        if i not in unique_list:
            unique_list.append(i)
            
unique_list

In [ ]:
data_features = data.copy()    

In [ ]:
data_features.head()

In [ ]:
for i in unique_list:
    data_features.insert(loc=9,
    column=i,
    value = 0)

In [ ]:
col = data_features['Features']
for idx, arr in enumerate(col):
    print(idx)
    for x in arr:
        for y in unique_list:
            if x == y: 
                data_features[y][idx] = 1

In [ ]:
data_features

## Challenges
- Having to go to each page in order to get the necessary data (takes longer time)
- showers, parking, and bedrooms have values with 0.5 increments and for those with more than 5, it's just placed as 5+

## Conclusion
- [Does it have personal identifiable information?]
- 